# Titanic data with decision trees

In [ ]:
import pandas as pd
titanic_data = pd.read_csv('titanic_data.csv')
titanic_data

In [ ]:
X = pd.DataFrame(titanic_data[features])
X

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
X['Sex'] = lb.fit_transform(X['Sex'])
X

In [ ]:
X['Embarked'] = lb.fit_transform(X['Embarked'])
X

In [ ]:
X['Categorized_Age'] = lb.fit_transform(X['Categorized_Age'])

In [ ]:
X = X.drop("Cabin", axis=1)
X = X.drop("Ticket", axis=1)

In [ ]:
X['Title'] = lb.fit_transform(X['Title'])

In [ ]:
y = pd.DataFrame(titanic_data['Survived'])

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X,y)
dt.score(X,y)

In [ ]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(dt, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [ ]:
from sklearn.datasets import load_iris
from sklearn import tree
iris = load_iris()
clf = tree.DecisionTreeClassifier()
clf = clf.fit(iris.data, iris.target)

In [ ]:
import graphviz 
dot_data = dt.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("dt") 

In [ ]:
titanic_data.column_names()

In [ ]:
features = [f for f in titanic_data.column_names() if f != 'Name' and f != 'Survived' and f!= 'PassengerId']
features

In [ ]:
for f in features:
    print (f, type(titanic_data[f][0]))

In [ ]:
def gini(l):
    # Returns gini index of a list of values
    print (1 - np.dot(l,l)*1.0/(sum(l))**2)
def entropy(l):
    # Returns the entropy of a list of values
    return np.dot(l, np.log2(l))/sum(l)

def classes_count(feature):
    # Given a column with discrete fields, it returns the dictionary
    # containing the fields and their number of appearances
    column = titanic_data[feature]
    classes = {}
    for element in column:
        if element in classes.keys():
            classes[element] += 1
        else:
            classes[element] = 1
    return classes

def pick_highest_class(column):
    # Given a column, it returns the feature that splits the column
    # with the highest gini impurity, and the two SFrames split by that feature
    

In [ ]:
[classes_count('Title')[key] for key in classes_count('Title')]

In [ ]:
titanic_data['Fare'].show()

In [ ]:
titanic_data[features]

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
X = [['Male', 1], ['Female', 3], ['Female', 2]]
enc.fit(X)

OneHotEncoder(categorical_features=None, categories=None,
    #drop=None,
    #dtype=<'numpy.float64'>,
    handle_unknown='ignore',
    n_values=None, sparse=True)
enc.categories_
#[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]
enc.transform([['Female', 1], ['Male', 4]]).toarray()
#array([[1., 0., 1., 0., 0.],
#       [0., 1., 0., 0., 0.]])
enc.inverse_transform([[0, 1, 1, 0, 0], [0, 0, 0, 1, 0]])
#array([['Male', 1],
#       [None, 2]], dtype=object)
enc.get_feature_names()
#array(['x0_Female', 'x0_Male', 'x1_1', 'x1_2', 'x1_3'], dtype=object)
drop_enc = OneHotEncoder(drop='first').fit(X)
drop_enc.categories_
#[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]
drop_enc.transform([['Female', 1], ['Male', 2]]).toarray()
#array([[0., 0., 0.],
#       [1., 1., 0.]])

In [ ]:
OneHotEncoder(categorical_features=None, categories=None, handle_unknown='ignore', n_values=None, sparse=True)

In [ ]:
np.array(enc.transform([['Female', 1], ['Male', 4]]))

# Kaggle walkthrough
https://www.kaggle.com/nhlr21/complete-titanic-tutorial-with-ml-nn-ensembling

In [ ]:
import pydot
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz, DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.exceptions import NotFittedError

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

from IPython.display import display


In [ ]:
path_train = './train.csv'
path_test = './test.csv'

In [ ]:
train_df_raw = pd.read_csv(path_train)
train_df_raw.head()

In [ ]:
train_df_raw.describe()

In [ ]:
train_df_raw.info()

In [ ]:
figure, axes = plt.subplots(1,1,figsize=(20, 6))
plot = sns.catplot(x="Embarked", y="Fare", hue="Sex", data=train_df_raw, palette=('Set1'), kind="bar", ax=axes)
plt.close(plot.fig)
plt.show()
display(train_df_raw[train_df_raw['Embarked'].isnull()])

In [ ]:
# Let's plot some histograms to have a previzualisation of some of the data ...
train_df_raw.drop(['PassengerId'], 1).hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
def preprocess_data(df):
    
    processed_df = df
        
    ########## Deal with missing values ##########
    
    # As we saw before, the two missing values for embarked columns can be replaced by 'C' (Cherbourg)
    processed_df['Embarked'].fillna('C', inplace=True)
    
    # We replace missing ages by the mean age of passengers who belong to the same group of class/sex/family
    processed_df['Age'] = processed_df.groupby(['Pclass','Sex','Parch','SibSp'])['Age'].transform(lambda x: x.fillna(x.mean()))
    processed_df['Age'] = processed_df.groupby(['Pclass','Sex','Parch'])['Age'].transform(lambda x: x.fillna(x.mean()))
    processed_df['Age'] = processed_df.groupby(['Pclass','Sex'])['Age'].transform(lambda x: x.fillna(x.mean()))
    
    # We replace the only missing fare value for test dataset and the missing values of the cabin column
    processed_df['Fare'] = processed_df['Fare'].interpolate()
    processed_df['Cabin'].fillna('U', inplace=True)
    
    ########## Feature engineering on columns ##########
    
    # Create a Title column from name column
    processed_df['Title'] = pd.Series((name.split('.')[0].split(',')[1].strip() for name in train_df_raw['Name']), index=train_df_raw.index)
    processed_df['Title'] = processed_df['Title'].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    processed_df['Title'] = processed_df['Title'].replace(['Mlle', 'Ms'], 'Miss')
    processed_df['Title'] = processed_df['Title'].replace('Mme', 'Mrs')
    processed_df['Title'] = processed_df['Title'].map({"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5})
    
    # Filling Age missing values with mean age of passengers who have the same title
    processed_df['Age'] = processed_df.groupby(['Title'])['Age'].transform(lambda x: x.fillna(x.mean()))

    # Transform categorical variables to numeric variables
    processed_df['Sex'] = processed_df['Sex'].map({'male': 0, 'female': 1})
    processed_df['Embarked'] = processed_df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
    
    # Create a Family Size, Is Alone, Child and Mother columns
    processed_df['FamillySize'] = processed_df['SibSp'] + processed_df['Parch'] + 1
    processed_df['FamillySize'][processed_df['FamillySize'].between(1, 5, inclusive=False)] = 2
    processed_df['FamillySize'][processed_df['FamillySize']>5] = 3
    processed_df['IsAlone'] = np.where(processed_df['FamillySize']!=1, 0, 1)
    processed_df['IsChild'] = processed_df['Age'] < 18
    processed_df['IsChild'] = processed_df['IsChild'].astype(int)
    
    # Modification of cabin column to keep only the letter contained corresponding to the deck of the boat
    processed_df['Cabin'] = processed_df['Cabin'].str[:1]
    processed_df['Cabin'] = processed_df['Cabin'].map({cabin: p for p, cabin in enumerate(set(cab for cab in processed_df['Cabin']))})
    
    # Create a ticket survivor column which is set to 1 if an other passenger with the same ticket survived and 0 else
    # Note : this implementation is ugly and unefficient, if sombody found a way to do it easily with pandas (it must be a way), please comment the kernel with your solution !
    processed_df['TicketSurvivor'] = pd.Series(0, index=processed_df.index)
    tickets = processed_df['Ticket'].value_counts().to_dict()
    for t, occ in tickets.items():
        if occ != 1:
            table = train_df_raw['Survived'][train_df_raw['Ticket'] == t]
            if sum(table) != 0:
                processed_df['TicketSurvivor'][processed_df['Ticket'] == t] = 1
    
    # These two columns are not useful anymore
    processed_df = processed_df.drop(['Name', 'Ticket', 'PassengerId'], 1)    
    
    return processed_df

In [ ]:
# Let's divide the train dataset in two datasets to evaluate perfomance of the machine learning models we'll use
train_df = train_df_raw.copy()
X = train_df.drop(['Survived'], 1)
Y = train_df['Survived']

X = preprocess_data(X)
# We scale our data, it is essential for a smooth working of the models. Scaling means that each columns as a 0 mean and a 1 variance
sc = StandardScaler()
X = pd.DataFrame(sc.fit_transform(X.values), index=X.index, columns=X.columns)
    
# Split dataset for model testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train.head()

In [ ]:
dt = DecisionTreeClassifier(min_samples_split=15, min_samples_leaf=20, random_state=42)
dt.fit(X_train, Y_train)
dt_prediction = dt.predict(X_test)

score = metrics.accuracy_score(Y_test, dt_prediction)
display_confusion_matrix(Y_test, dt_prediction, score=score)

In [ ]:
processed_data = titanic_data.remove_columns(['Name', 'Ticket', 'Cabin', 'Age'])
processed_data

# App dataset

In [ ]:
app_dataset = pd.DataFrame({
    'Gender': ['Female','Female','Male','Female','Male','Male'],
    'Age': [15, 25, 32, 35, 12, 14],
    'App': ['Atom Count', 'Check Mate Mate', 'Beehive Finder', 'Check Mate Mate', 'Atom Count', 'Atom Count']})
app_dataset

In [ ]:
app_dataset_one_hot = pd.DataFrame(
    {'Gender_Female':[1,1,0,1,0,0],
     'Gender_Male':[0,0,1,0,1,1],
     'Age_Young':[1,0,0,0,1,1],
     'Age_Old':[0,1,1,1,0,0],
     'App_Atom_Count':[1,0,0,0,1,1],
     'App_Beehive_Finder':[0,0,1,0,0,0],
     'App_Check_Mate_Mate':[0,1,0,1,0,0]})
app_dataset_one_hot

In [ ]:
X = app_dataset_one_hot[['Gender_Female','Gender_Male','Age_Old','Age_Young']]
y = app_dataset_one_hot[['App_Atom_Count','App_Beehive_Finder','App_Check_Mate_Mate']]

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X,y)
dt.score(X,y)

In [ ]:
X

In [ ]:
y

In [ ]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(dt, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [ ]:
app_dataset = pd.DataFrame({
    'Gender_Female':[1,1,0,1,0,0],
    'Gender_Male':[0,0,1,0,1,1],
    'Age': [15, 25, 32, 35, 12, 14],
    'App': ['Atom Count', 'Check Mate Mate', 'Beehive Finder', 'Check Mate Mate', 'Atom Count', 'Atom Count']})
print(app_dataset)

In [ ]:
features = app_dataset_new[['Age','Gender_Female','Gender_Male']]
print(features)

In [ ]:
print(labels)

In [ ]:
app_model = DecisionTreeClassifier()
app_model.fit(features, labels)
app_model.score(features, labels)

In [ ]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(new_dt, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

# Gini index calculation

In [ ]:
from __future__ import division

In [ ]:
elements = ['A', 'A', 'A', 'C', 'B', 'C']
def counts(elements):
    classes = {}
    for element in elements:
        if element in classes:
            classes[element] += 1
        else:
            classes[element] = 1
    return [classes[e] for e in classes]
counts(elements)

In [ ]:
def gini(counts):
    n = sum(counts)
    return 1 - sum([p_i**2/n**2 for p_i in counts])
gini(counts(elements))

In [ ]:
# Splitting age on 13
gini(counts(elements)) - (gini([1])*1/6 + gini([2,1,2])*5/6)

In [ ]:
# Splitting age on 14.5
gini(counts(elements)) - (gini([2])*2/6 + gini([1,1,2])*4/6)

In [ ]:
# Splitting age on 20
gini(counts(elements)) - (gini([3])*3/6 + gini([1,2])*3/6)

In [ ]:
# Splitting age on 27
gini(counts(elements)) - (gini([3,1])*4/6 + gini([1,1])*2/6)

In [ ]:
# Splitting age on 33
gini(counts(elements)) - (gini([3,1,1])*5/6 + gini([1])*1/6)

# Spam example

In [ ]:
spam_dataset = pd.DataFrame({
    'Lottery':[7,3,9,1,2,4,1,3,6,7,8,9],
    'Sale':[1,2,3,3,6,7,9,10,5,8,4,6],
    'Spam': ['no','no','no','no','no','no','yes','yes','yes','yes','yes','yes']})
spam_dataset

In [ ]:
features = spam_dataset[['Lottery', 'Sale']]
labels = spam_dataset['Spam']
spam_decision_tree = DecisionTreeClassifier()
spam_decision_tree.fit(X,y)
spam_decision_tree.score(X,y)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.scatter(X['Lottery'][:6], X['Sale'][:6])
plt.scatter(X['Lottery'][6:], X['Sale'][6:])

In [ ]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(spam_decision_tree, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())